First, we start off with importing any necessary libraries. 

In [19]:

import cv2 
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random

In [20]:
##  BIG MICH

img = cv2.imread('leaves.jpg')

cv2.imshow("image Leaves ", img)

imgHSV= cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
imgHue = imgHSV[:, :, 0]
imgSat = imgHSV[:, :, 1]
imgVal= imgHSV[:, :, 2]

cv2.imshow('HSV Image', imgHSV)
cv2.imshow('Hue Channel', imgHue )
cv2.imshow('Saturation Channel', imgSat)
cv2.imshow('Value Channel', imgVal)

we gotta convert that to subplots


With the image read and converted to HSV format, the next step is to apply a mask of threshold values. 

In [21]:
#waits for user to press a key
cv2.waitKey(0)
cv2.destroyAllWindows()